In [8]:
## This notebook provides a simple demonstration on how to use the codebase 
from helper import convert_raw_json
from src.estimation import estimate_text_distribution
from src.MLE import MLE
import json 
import pandas as pd 

In [5]:
base_json_dir = "data" # the path to the raw data 
subject = "medicine"
year = 2024
month = 4
output_folder = "{}/{}".format("demo_output", subject)
output_name = "{}_{}".format(year, month)

In [6]:
#load the json file
with open("{}/{}.json".format(base_json_dir, year), "r") as f:
    source_json = json.load(f)

# the entire collection of data in the source json file 
all_collection = convert_raw_json(source_json) 

# articles related to the given subject 
subj_collection = all_collection.get_articles_subject(subject)

# articles related to the given subject published at the given time 
subj_time_collection = subj_collection.get_articles_time(year, month)
subj_time_collection.assign_subject(subject)

In [7]:
# export the data as a parquet 
subj_time_collection.export_parquet(output_folder, output_name)

Exporting parquet


100%|███████████████████████████████████████| 8335/8335 [05:06<00:00, 27.21it/s]


File Exported as: demo_output/medicine/2024_4.parquet


In [10]:
dist_loc = "data/distribution/distribution.parquet" 
dist_df = pd.read_parquet(dist_loc)
dist_df

,Word,logP,log1-P,logQ,log1-Q
0,delve,-9.843921,-0.000053,-5.937896,-0.002641
1,delves,-9.150774,-0.000106,-5.473278,-0.004206
2,encompass,-9.843921,-0.000053,-6.350124,-0.001748
3,underscore,-10.162375,-0.000039,-6.736685,-0.001187
4,shedding,-9.939231,-0.000048,-6.552492,-0.001428
...,...,...,...,...,...
12265,usually,-7.065666,-0.000854,-11.918468,-0.000007
12266,devoted,-7.010708,-0.000903,-11.918468,-0.000007
12267,lot,-6.610604,-0.001347,-11.918468,-0.000007
12268,meanwhile,-5.772566,-0.003117,-11.407643,-0.000011


In [14]:
output_data = pd.read_parquet("{}/{}.parquet".format(output_folder, output_name))
output_data

,inference_sentence
0,"[temporomandibular, disorder, is, a, disorder,..."
1,"[the, disorders, can, disturb, daily, activity..."
2,"[the, therapy, consuming, time, and, cost]"
3,"[early, detection, of, temporomandibular, diso..."
4,"[increased, cortisol, can, be, found, in, myof..."
...,...
107166,"[objective, to, analyze, if, the, level, of, a..."
107167,"[materials, and, methods, narrative, review, o..."
107168,"[the, research, obeyed, the, criterion, of, ye..."
107169,"[conclusion, for, a, process, of, gluteus, max..."


In [11]:
# estimate the fraction of text written by GPT 
model= MLE(dist_loc)
alpha, ci= model.inference("{}/{}.parquet".format(output_folder, output_name))

In [12]:
print("alpha:{}, CI: {}".format(alpha, ci)

0.031

In [13]:
ci

0.002